In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch
import tensorflow as tf
print(torch.cuda.is_available())
import pandas as pd
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split


In [2]:
import pandas as pd
import json
import time
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer

# Load datasets
def load_datasets():
    # Replace with your actual dataset paths
    data1 = pd.read_csv('C:\Final Project in Uni\CTGAN\expe\infogandataset/2023/final_2023_dataset.csv')
    data2 = pd.read_csv('C:\Final Project in Uni\CTGAN\expe/realdata/2023\Clean_2023data.csv')
    return data1, data2

# Preprocess datasets
def preprocess_data(data, target_column):
    # Convert target column from string to numeric
    label_encoder = LabelEncoder()
    data[target_column] = label_encoder.fit_transform(data[target_column])
    
    X = data.drop(columns=[target_column])
    y = data[target_column]
    return X, y, label_encoder

# Stratified sampling by class percentage
def stratified_sampling(X, y, sample_percentage):
    data = pd.concat([X, y], axis=1)
    sampled_data = data.groupby(y.name, group_keys=False).apply(lambda x: x.sample(frac=sample_percentage, random_state=42))
    X_sampled = sampled_data.iloc[:, :-1]
    y_sampled = sampled_data.iloc[:, -1]
    return X_sampled, y_sampled

# Build MLP model
def build_mlp(input_dim, n_classes):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dense(64, activation='relu'),
        Dense(n_classes, activation='softmax')  # Softmax for multi-class classification
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy',  # For integer labels
                  metrics=['accuracy'])
    return model

# Evaluate model
def evaluate_model(model, X_test, y_test, n_classes, is_keras=False):
    if is_keras:
        y_pred_probs = model.predict(X_test)
        y_pred = y_pred_probs.argmax(axis=1)
    else:
        y_pred = model.predict(X_test)
        y_pred_probs = model.predict_proba(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    if n_classes > 2:
        lb = LabelBinarizer()
        lb.fit(range(n_classes))  # Ensure all classes are included
        y_test_bin = lb.transform(y_test)  # Binarize y_test
        auc = roc_auc_score(y_test_bin, y_pred_probs, multi_class='ovr', average='weighted')
    else:
        auc = roc_auc_score(y_test, y_pred_probs[:, 1]) if n_classes == 2 else None

# Save results to log
def log_results(log_file, model_name, train_dataset, test_dataset, metrics, train_time):
    log_entry = {
        "model": model_name,
        "train_dataset": train_dataset,
        "test_dataset": test_dataset,
        "metrics": metrics,
        "train_time": train_time
    }
    with open(log_file, 'a') as f:
        f.write(json.dumps(log_entry) + '\n')

In [ ]:
target_column = 'Target'  # Replace with your target column
log_file = 'C:\Final Project in Uni\CTGAN\expe\infogandataset/2023experiment_log_infogan2023.json'
sample_percentages = [0.1, 0.2, 0.4, 0.8]  # Percentages for stratified sampling
    
    # Load datasets
data1, data2 = load_datasets()
X1, y1, label_encoder1 = preprocess_data(data1, target_column)
X2, y2, label_encoder2 = preprocess_data(data2, target_column)
n_classes = len(label_encoder1.classes_)  # Number of classes
    
    # Models to train
models = {
        "Random Forest": RandomForestClassifier(random_state=42),
        "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'),
        "LightGBM": LGBMClassifier(random_state=42),
        "MLP": None  # Special handling for MLP
    }

    # Loop through sample percentages
for sample_percentage in sample_percentages:
    print(f"Running experiments with {int(sample_percentage * 100)}% of the data per class...")
    
    # Stratified sampling
    X1_sampled, y1_sampled = stratified_sampling(X1, y1, sample_percentage)
    #X2_sampled, y2_sampled = stratified_sampling(X2, y2, sample_percentage)

    # Split dataset 2 into train/test for evaluation
    X2_train, X2_test, y2_train, y2_test = train_test_split(
        X2, y2, test_size=0.2, random_state=42
    )
    
    # Scale data for MLP
    scaler1 = StandardScaler()
    scaler2 = StandardScaler()
    #X1_scaled = scaler1.fit_transform(X1_sampled) 
    X1_scaled = X1_sampled
    X2_train_scaled = scaler2.fit_transform(X2_train)
    X2_test_scaled = scaler2.transform(X2_test)

    # **Train on dataset 1 and test on test split of dataset 2**
    for model_name, model in models.items():
        print(f"Training {model_name} on dataset1_{int(sample_percentage * 100)}% and testing on dataset2_{int(sample_percentage * 100)}% (test split)")
        start_time = time.time()  # Start timer
        if model_name == "MLP":
            mlp = build_mlp(X1_scaled.shape[1], n_classes)
            mlp.fit(X1_scaled, y1_sampled, epochs=10, batch_size=32, verbose=0)
            train_time = time.time() - start_time  # End timer
            metrics = evaluate_model(mlp, X2_test_scaled, y2_test, n_classes, is_keras=True)
        else:
            model.fit(X1_sampled, y1_sampled)
            train_time = time.time() - start_time  # End timer
            metrics = evaluate_model(model, X2_test_scaled, y2_test, n_classes)
        log_results(log_file, model_name, f"info2023_{int(sample_percentage * 100)}%", f"realdata2023_{int(sample_percentage * 100)}% (test split)", metrics, train_time)

    # **Train and test entirely on dataset 2 (train/test split)**
    # for model_name, model in models.items():
    #     print(f"Training and testing {model_name} entirely on dataset2_{int(sample_percentage * 100)}% (train/test split)")
    #     start_time = time.time()  # Start timer
    #     if model_name == "MLP":
    #         mlp = build_mlp(X2_train_scaled.shape[1], n_classes)
    #         mlp.fit(X2_train_scaled, y2_train, epochs=10, batch_size=32, verbose=0)
    #         train_time = time.time() - start_time  # End timer
    #         metrics = evaluate_model(mlp, X2_test, y2_test, n_classes, is_keras=True)
    #     else:
    #         model.fit(X2_train, y2_train)
    #         train_time = time.time() - start_time  # End timer
    #         metrics = evaluate_model(model, X2_test, y2_test, n_classes)
    #     log_results(log_file, model_name, f"ctabgan20_{int(sample_percentage * 100)}% (train)", f"realdata20_{int(sample_percentage * 100)}% (test)", metrics, train_time)

print(f"Experiment results with multiclass classification logged in {log_file}")